In [25]:
import pandas as pd
df = pd.DataFrame({'Naive' : [Naive], 'svm':[svm], 'RF':[RF], 'DT':[DT], 'VT':[VT]})
df


,Naive,svm,RF,DT,VT
0,0.776118,0.695035,0.50769,0.482873,0.723579


In [2]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB 
from sklearn.metrics import accuracy_score
from sklearn.datasets import fetch_20newsgroups

train_news= fetch_20newsgroups(subset='train', remove=('headers', 'footers', 'quotes'), random_state=156)
X_train = train_news.data
y_train = train_news.target

test_news=fetch_20newsgroups(subset='test', remove=('headers', 'footers', 'quotes'), random_state=156)
X_test = test_news.data
y_test = test_news.target

tfidf_vect = TfidfVectorizer(stop_words = 'english', ngram_range=(1, 2), max_df = 300)
tfidf_vect.fit(X_train)

X_train_tfidf_vect = tfidf_vect.transform(X_train)
X_test_tfidf_vect = tfidf_vect.transform(X_test)



tfidf_vect = TfidfVectorizer()
tfidf_vect.fit(X_train)
X_train_tfidf_vect = tfidf_vect.transform(X_train)
X_test_tfidf_vect = tfidf_vect.transform(X_test)


In [3]:
tfidf_vect = TfidfVectorizer(stop_words = 'english', ngram_range=(1, 2), max_df = 300)
tfidf_vect.fit(X_train)
X_train_tfidf_vect = tfidf_vect.transform(X_train)
X_test_tfidf_vect = tfidf_vect.transform(X_test)

In [11]:
import numpy as np

mod = MultinomialNB()
mod.fit(X_train_tfidf_vect, y_train)

param_grid = [{'alpha': np.linspace(0.01, 1, 100)}]
gs = GridSearchCV(estimator=mod, param_grid=param_grid, scoring='accuracy', cv=5, n_jobs=-1)
gs.fit(X_train_tfidf_vect, y_train)

print('최적 하이퍼 파라미터:\n', gs.best_params_)
print('최고 예측 정확도: {0:.4f}'.format(gs.best_score_))

최적 하이퍼 파라미터:
 {'alpha': 0.02}
최고 예측 정확도: 0.7761


In [14]:
Naive = gs.best_score_
Naive

0.7761182174472435

In [8]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn import svm
from sklearn.model_selection import GridSearchCV, StratifiedKFold, train_test_split

clf = svm.SVC(kernel='linear')
clf.fit(X_train_tfidf_vect, y_train)

pred = clf.predict(X_test_tfidf_vect) #테스트 데이터에 대한 예측
print("정확도:", accuracy_score(y_test, pred)) #예측값과 실제값 비교

정확도: 0.6950345193839618


In [9]:
svm = accuracy_score(y_test, pred)
svm

0.6950345193839618

In [15]:
from sklearn.ensemble import RandomForestClassifier

rf_clf = RandomForestClassifier(random_state=0)
rf_clf.fit(X_train_tfidf_vect, y_train)
pred = rf_clf.predict(X_test_tfidf_vect)
print("정확도:", accuracy_score(y_test, pred))

정확도: 0.6344928305894849


In [16]:
RF = accuracy_score(y_test, pred)
RF

0.6344928305894849

In [17]:
from sklearn.tree import DecisionTreeClassifier

dcl = DecisionTreeClassifier()
dcl.fit(X_train_tfidf_vect, y_train)
pred = dcl.predict(X_test_tfidf_vect)
accuracy_score(y_test, pred)

0.48287307488050984

In [18]:
DT = accuracy_score(y_test, pred)
DT

0.48287307488050984

In [19]:
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import VotingClassifier

lr_clf = LogisticRegression(C=10)
mod = MultinomialNB(alpha=0.02)

# 개별 모델을 소프트 보팅 기반의 앙상블 모델로 구현한 분류기
vo_clf = VotingClassifier(estimators=[('LR', lr_clf), ('MNB', mod)], voting='soft')

#VotingClassifier 학습/예측/평가.
vo_clf.fit(X_train_tfidf_vect, y_train)
pred = vo_clf.predict(X_test_tfidf_vect)
print('Voting 분류기 정확도: {0:.4f}'.format(accuracy_score(y_test, pred)))

/home/prlhspt/anaconda3/envs/venv/lib/python3.6/site-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


Voting 분류기 정확도: 0.7236


In [20]:
VT = accuracy_score(y_test, pred)
VT

0.723579394583112

In [21]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV

params = {
    'n_estimators':[100],
    'max_depth' : [6, 8, 10, 12], 
    'min_samples_leaf' : [8, 12, 18 ],
    'min_samples_split' : [8, 16, 20]
}
# RandomForestClassifier 객체 생성 후 GridSearchCV 수행
rf_clf = RandomForestClassifier(n_estimators=300, max_depth=10, min_samples_leaf=8, min_samples_split=8)
                                
rf_clf.fit(X_train_tfidf_vect, y_train)

gs = GridSearchCV(rf_clf , param_grid=params , cv=2, n_jobs=-1 )
gs.fit(X_train_tfidf_vect, y_train)

print('최적 하이퍼 파라미터:\n', gs.best_params_)
print('최고 예측 정확도: {0:.4f}'.format(gs.best_score_))

최적 하이퍼 파라미터:
 {'max_depth': 12, 'min_samples_leaf': 8, 'min_samples_split': 8, 'n_estimators': 100}
최고 예측 정확도: 0.5077


In [23]:
RF = gs.best_score_
RF

0.5076895881209121

In [24]:
import pandas as pd
df = pd.DataFrame({'Naive' : [Naive], 'svm':[svm], 'RF':[RF], 'DT':[DT], 'VT':[VT]})# , 'SVC' : y_pred_sv, 'VT' : y_pred_dcl})
df


,Naive,svm,RF,DT,VT
0,0.776118,0.695035,0.50769,0.482873,0.723579
